<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gather-Follower-&amp;-Statuses-Counts-for-Podcasts-with-Twitter-Accounts" data-toc-modified-id="Gather-Follower-&amp;-Statuses-Counts-for-Podcasts-with-Twitter-Accounts-1">Gather Follower &amp; Statuses Counts for Podcasts with Twitter Accounts</a></span><ul class="toc-item"><li><span><a href="#Goal-is-to-use-these-to-enrich-our-understanding-of-podcast-social-activity." data-toc-modified-id="Goal-is-to-use-these-to-enrich-our-understanding-of-podcast-social-activity.-1.1">Goal is to use these to enrich our understanding of podcast social activity.</a></span></li></ul></li></ul></div>

In [1]:
import pickle

import features

import time
import re

import twitter
api = twitter.Api(consumer_key='3ExAhrEEf6c1zhtFzswFJ7cGD',
  consumer_secret='tnyOfCRVq4YT6uCo6IVChmQHCX3iRTK1Yg45ZqWqAW2EjUQwKO',
  access_token_key='1166032070667202565-9MbVIOwbCC8wnVarMsIE2NkBDwADix',
  access_token_secret='OpCglzB7WsTyeEjI7CeNpo1cHTJSYQqKe4ylOrQpSBlSx')

In [2]:
print(api.VerifyCredentials())

{"created_at": "Mon Aug 26 16:59:13 +0000 2019", "default_profile": true, "description": "data scientist | meditator | Coloradan in BK/NYC", "favourites_count": 1, "followers_count": 4, "friends_count": 20, "id": 1166032070667202565, "id_str": "1166032070667202565", "name": "Thomas Kavanagh", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1166032951932403714/lkQcnMwS_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1166032951932403714/lkQcnMwS_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ThomasK26635081"}


## Gather Follower & Statuses Counts for Podcasts with Twitter Accounts

### Goal is to use these to enrich our understanding of podcast social activity.

Tutorial found here:

https://www.alexkras.com/how-to-get-user-feed-with-twitter-api-and-python/

We will unload (any) version of the sanitized dataframe, in order to gather a list of channels in order to create a dictionary.

In [3]:
df_version = 'merged_10-06_23-26.pickle'
import_path = '../scraped/merged/' + df_version
with open(import_path, 'rb') as file:
    raw_df = pickle.load(file)

raw_df['twitter'] = raw_df['ch_feed-socials'].apply(lambda row: features.has_domain(row, 'twitter'))

In [4]:
t_cols = ['title','chan_url','ch_feed-socials','twitter']

In [5]:
raw_df.sample(3)[t_cols]

,title,chan_url,ch_feed-socials,twitter
190,Dance Shark Podcast,https://castbox.fm/channel/Dance-Shark-Podcast...,"[https://twitter.com/DanceSharkPod, https://ww...",1
17,The Secret History of the Future,https://castbox.fm/channel/The-Secret-History-...,[https://slate.com],0
107,Collegehood Advice - How to succeed and thrive...,https://castbox.fm/channel/Collegehood-Advice-...,[https://collegehoodadvice.com/],0


In [6]:
raw_df[raw_df.twitter == 1].sample(3)[t_cols]

,title,chan_url,ch_feed-socials,twitter
136,Duncan Trussell Family Hour,https://castbox.fm/channel/Duncan-Trussell-Fam...,"[https://twitter.com/duncantrussell, https://w...",1
25,Your Morning Basket,https://castbox.fm/channel/Your-Morning-Basket...,"[https://twitter.com/hspambarnhill, https://pa...",1
61,TWILA Aftershow,https://castbox.fm/channel/TWILA-Aftershow-id2...,"[https://twitter.com/twilatv, https://www.face...",1


In [7]:
twitter_chans = raw_df[raw_df.twitter == 1][['title','chan_url','ch_feed-socials']].to_dict(orient='records')

In [8]:
def extract_twitter_screen_name(ch_feed_socials):

    if len(ch_feed_socials) == 0:
        return ''
    else:
        pattern = '.+(twitter.com/)(?P<screen_name>.+)'
        for url in ch_feed_socials:
#             print(url)
            try:
                screen_name = re.match(pattern, url).group('screen_name')
                if screen_name:
                    return screen_name
            except:
                oass
        return ''

In [15]:
with open('../social_metrics/twitter/channel_stats_by_name_oct6_11p_copy.pickle', 'rb') as file:
    twitter_dict = pickle.load(file)

In [16]:
for chan in twitter_dict.keys():
    if twitter_dict[chan]['screen_name']:
        continue
    else:
        twitter_dict[chan]['screen_name'] = extract_twitter_screen_name(twitter_dict[chan]['ch_feed-socials'])

In [14]:
len(twitter_dict)

728

Let's create a set of unique twitter handles, as its own dataframe, which we will use as a reference.

In [100]:
# with open('../social_metrics/twitter/channel_stats_by_name.pickle', 'rb') as file:
#     twitter_follower_dict = pickle.load(file)

In [17]:
def add_channels_to_twitter_dictionary(channnels, output_dict):
    for chan in channnels:
        key = chan['title']
        try:
            output_dict[key].update(chan)
        except:
            output_dict[key] = chan
    return True

In [25]:
twitter_chans[:5]

[{'title': 'Dose of Leadership with Richard Rierson | Authentic & Courageous Leadership Development',
  'chan_url': 'https://castbox.fm/channel/Dose-of-Leadership-with-Richard-Rierson-|-Authentic-%26-Courageous-Leadership-Development-id2043375',
  'ch_feed-socials': ['https://twitter.com/DoseOfLeader',
   'https://www.doseofleadership.com']},
 {'title': 'Read to Lead Podcast',
  'chan_url': 'https://castbox.fm/channel/Read-to-Lead-Podcast-id1300385',
  'ch_feed-socials': ['https://twitter.com/THEjeffbrown',
   'https://readtoleadpodcast.com']},
 {'title': 'Business Bootcamp',
  'chan_url': 'https://castbox.fm/channel/Business-Bootcamp-id8156',
  'ch_feed-socials': ['https://twitter.com/thebizpodcast',
   'https://www.facebook.com/businessbootcamppodcast',
   'https://businessbootcamppodcast.com']},
 {'title': 'The Speaking Club: Mastering the Art of Public Speaking',
  'chan_url': 'https://castbox.fm/channel/The-Speaking-Club%3A-Mastering-the-Art-of-Public-Speaking-id1365451',
  'ch_fe

In [26]:
add_channels_to_twitter_dictionary(twitter_chans, twitter_dict)

True

In [27]:
[(k, twitter_dict[k]) for k in list(twitter_dict.keys())[:5]]

[('Small Town Horror',
  {'title': 'Small Town Horror',
   'chan_url': 'https://castbox.fm/channel/Small-Town-Horror-id1266396',
   'ch_feed-socials': ['https://twitter.com/SmallTwnHorror',
    'https://www.facebook.com/smalltownhorrorpodcast',
    'https://play.acast.com/s/smalltownhorror'],
   'screen_name': 'SmallTwnHorror',
   'follower_count': 0}),
 ('I Should Be Writing',
  {'title': 'I Should Be Writing',
   'chan_url': 'https://castbox.fm/channel/I-Should-Be-Writing-id210',
   'ch_feed-socials': ['https://twitter.com/mightymur',
    'http://murverse.com/tag/isbw/'],
   'screen_name': 'mightymur',
   'follower_count': 14970}),
 ('Book Riot - The Podcast',
  {'title': 'Book Riot - The Podcast',
   'chan_url': 'https://castbox.fm/channel/Book-Riot-The-Podcast-id1207097',
   'ch_feed-socials': ['https://twitter.com/BookRiot',
    'https://bookriot.com/listen/shows/thepodcast/'],
   'screen_name': 'BookRiot',
   'follower_count': 214860}),
 ('The Disney Story Origins Podcast',
  {'t

In [28]:
def update_screen_names(t_dict):
    
    chan_list = t_dict.keys()
    
    for chan in chan_list:
        if 'screen_name' in t_dict[chan].keys():
            pass
        else:
            socials = t_dict[chan]['ch_feed-socials']
            t_dict[chan]['screen_name'] = extract_twitter_screen_name(socials)
    
    return True

In [29]:
update_screen_names(twitter_dict)

True

In [25]:
[(k, twitter_dict[k]) for k in list(twitter_dict.keys())[:5]]

[('Small Town Horror',
  {'title': 'Small Town Horror',
   'chan_url': 'https://castbox.fm/channel/Small-Town-Horror-id1266396',
   'ch_feed-socials': ['https://twitter.com/SmallTwnHorror',
    'https://www.facebook.com/smalltownhorrorpodcast',
    'https://play.acast.com/s/smalltownhorror'],
   'screen_name': 'SmallTwnHorror'}),
 ('I Should Be Writing',
  {'title': 'I Should Be Writing',
   'chan_url': 'https://castbox.fm/channel/I-Should-Be-Writing-id210',
   'ch_feed-socials': ['https://twitter.com/mightymur',
    'http://murverse.com/tag/isbw/'],
   'screen_name': 'mightymur'}),
 ('Book Riot - The Podcast',
  {'title': 'Book Riot - The Podcast',
   'chan_url': 'https://castbox.fm/channel/Book-Riot-The-Podcast-id1207097',
   'ch_feed-socials': ['https://twitter.com/BookRiot',
    'https://bookriot.com/listen/shows/thepodcast/'],
   'screen_name': 'BookRiot'}),
 ('The Disney Story Origins Podcast',
  {'title': 'The Disney Story Origins Podcast',
   'chan_url': 'https://castbox.fm/cha

In [30]:
def get_follower_counts(t_dict, api_calls=1):
    
    chan_list = t_dict.keys()
    
    for chan in chan_list:
        
#         time.sleep(3)

        if api_calls < 1:
            return 'local API limit reached'
        if 'screen_name' in t_dict[chan].keys():
            sn = t_dict[chan]['screen_name']
#             print(f'about to scan {chan} with sn {sn}')
            if 'follower_count' in t_dict[chan].keys():
#                 print('already calculated follower count for ',chan)
                continue
            else:
                try:
                    user = api.GetUser(screen_name=sn)
                    follower_count = user.followers_count

                    print(f'user {sn} has {follower_count} followers')
                    t_dict[chan]['follower_count'] = follower_count

                    api_calls = api_calls - 1

                except:
                    t_dict[chan]['follower_count'] = 0
                    print(f'failed to pull follower count for {chan}')
                    continue
        else:
            continue

In [31]:
get_follower_counts(twitter_dict, api_calls=4000)

user DoseOfLeader has 2003 followers
user THEjeffbrown has 4886 followers
user thebizpodcast has 276 followers
user saraharcher15 has 1204 followers
user loveleelara has 44 followers
user TanveerNaseer has 26443 followers
user dollrstodonuts has 1142 followers
user badass_agile has 653 followers
failed to pull follower count for Real Estate Investing For Cash Flow Hosted by Kevin Bupp.
user PayBalances has 4979 followers
user cannainsider has 5108 followers
user SocialRookie has 231 followers
user RBIPodcast has 5245 followers
user ricedelman has 16609 followers
user ClaytonMorris has 103572 followers
user thebtcpodcast has 9438 followers
user lisaabramowicz1 has 70694 followers
user theRealKiyosaki has 1260436 followers
user andrewhorowitz has 11297 followers
user whatbitcoindid has 5608 followers
user EpicREInvesting has 5249 followers
user realvision has 63422 followers
user acast has 9106 followers
user Flip2Freedom has 24126 followers
user InPennyStock has 3758 followers
user MTIP

user DonaldCKelly has 4073 followers
user dreamsindrive has 2303 followers
user TheDovBaron has 15018 followers
user Thrive_15 has 7287 followers
user RyanMoran has 6604 followers
user NewManPodcast has 1247 followers
user RevRealEstate_ has 556 followers
user techmoneytalks has 532 followers
user manujagg has 1715 followers
user BNI_official_pg has 24037 followers
user DigitalMktr has 46470 followers
user scottvoelker has 2012 followers
user magazine has 721 followers
user rickmulready has 5757 followers
user TheGrowthShow has 1099 followers
user mywifequit has 6167 followers
user CoachingKrista has 104 followers
user nscalice has 19433 followers
user ResumoCast has 1378 followers
user FurboPodcast has 290 followers
user joanboluda has 10862 followers
user nealschaffer has 221873 followers
user BrandonGaille has 24956 followers
user CriticalRole has 328483 followers
user cartalk has 45690 followers
user chmurph has 38789 followers
user giantbomb has 215152 followers
user glasscannonpo

In [36]:
list(twitter_dict.keys())[-20:]

['Ghost Stories, a Destiny Podcast',
 'Kotaku Splitscreen',
 'Bonfireside Chat - A Dark Souls and Bloodborne Podcast',
 'Evan and Katelyn Podcast',
 'The Gaming Hub: Your Home for Xbox, PlayStation, and Nintendo News and Opinions!',
 'The Brewing Network Presents | The Session',
 'The Homeschool Solutions Show with Pam Barnhill',
 'The Survivalist Prepper Podcast',
 'House of #EdTech',
 "Tech Talker's Quick and Dirty Tips to Navigate the Digital World",
 'All JavaScript Podcasts by Devchat.tv',
 'Fix It Home Improvement',
 'Better Podcasting',
 'The SavingsAngel Show',
 'College Admissions Decoded',
 'Podcasting Simplified',
 'The Art of World Building: Creating Breakout Fantasy and Science Fiction Worlds In Stories and Gaming',
 'Ask The Tech Guy (Video HD)',
 'WRITERS IN TECH',
 'Ask The Tech Guy (MP3)']

In [35]:
with open('../social_metrics/twitter/channel_stats_by_name_oct7_9p.pickle', 'wb') as file:
    pickle.dump(twitter_dict, file)

In [ ]:
twitter_dict[]